In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [4]:
dep_rates = pd.read_csv('C:\\my\\prog\\pet banks\\raw_data\\deposit_rates.csv', sep=',', encoding='utf-8')
monetary_rew = pd.read_csv('C:\\my\\prog\\pet banks\\raw_data\\monetary_review.csv', sep=',', encoding='utf-8')
money_mass = pd.read_csv('C:\\my\\prog\\pet banks\\raw_data\\money_mass.csv', sep=',', encoding='utf-8')

In [5]:
dep_rates.head(50)

,Ставки вознаграждения банков по привлеченным депозитам (по срокам и видам валют),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94
0,проценты за месяц,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,01.22,NaN,02.22,NaN,03.22,NaN,04.22,NaN,05.22,NaN,06.22,NaN,07.22,NaN,08.22,NaN,09.22,NaN,10.22,NaN,11.22,NaN,12.22,NaN,01.23,NaN,02.23,NaN,03.23,NaN,04.23,NaN,05.23,NaN,06.23,NaN,07.23,NaN,08.23,NaN,09.23,NaN,10.23,NaN,11.23,NaN,12.23,NaN,01.24,NaN,02.24,NaN,03.24,NaN,04.24,NaN,05.24,NaN,06.24,NaN,07.24,NaN,08.24,NaN,09.24,NaN,10.24,NaN,11.24,NaN,12.24,NaN,01.25,NaN,02.25,NaN,03.25,NaN,04.25,NaN,05.25,NaN,06.25,NaN,07.25,NaN,08.25,NaN,09.25,NaN,10.25,NaN,11.25,NaN
2,NaN,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ,KZT,СКВ
3,Депозиты небанковских юридических лиц,7.9,0.3,10.3,0.3,10.9,0.3,11.6,0.4,11.5,0.5,11.6,0.5,12.3,0.5,12.4,0.5,12.4,0.8,13.6,0.8,13.8,0.9,14.4,0.9,14.5,0.9,14.5,0.6,14.5,0.7,14.5,0.8,14.5,1.1,14.6,1.7,14.6,2.2,14.7,2.1,14.6,2.2,14.2,2.2,14.6,2.6,14.6,2.6,14.3,2.6,14,2.5,13.6,2.6,13.7,2.5,13.7,2.6,13.5,2.5,13.2,2.6,13.1,2.5,13.2,2.6,13.1,2.5,13.2,2.4,14,2.5,14.1,2.5,14.2,2.5,15,2.4,15.4,2.7,15.4,2.7,15.4,2.7,15.4,2.6,15.4,2.6,15.4,2.5,16.3,2.4,16.8,2.3
4,в том числе:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,до востребования,0.1,0.1,0.1,0,0.2,0.1,0.6,0.1,3.4,0,0.1,0.1,0.1,0,0.1,0,0.1,0,4.6,0,0.1,0,0.5,0,0.1,0.1,7,0.1,0.4,0.1,1.4,0.1,0.1,0.1,13.7,0.1,13.6,0.1,1.2,0.1,0.6,0.1,0,0.1,0.1,0.1,0.2,0.1,0,0.1,0.4,0.1,0.1,0.1,0.1,0.1,0,0.1,0,0.1,0.1,0.1,0.1,0.1,0.2,0.1,0.1,0.1,0.1,0.1,0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0,0.1,0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
6,условные,4.3,0,3.4,0.2,4.5,0.3,2.7,0.5,2.5,0.6,6.3,0.2,6.9,0.6,4.2,0.7,5.7,0.7,4,0.1,10,0.7,6.2,0.5,7.5,0.2,7.7,0.5,10.5,0.4,13.4,0.6,6.6,0.1,8.6,0.7,6.5,1.1,12.6,1.8,11.6,2.2,7.3,0.9,9.9,1.5,9.2,1.6,7.5,0.3,5.9,1.2,9.9,0.8,11.7,0.9,4.1,0.7,10.9,1.2,10.5,1,9.1,1.6,10.8,1.9,9.1,2,9,0.6,9.5,1.7,8.6,0.3,10.4,2,9.3,0.9,12.9,1.1,9,2.5

In [6]:
monetary_rew = monetary_rew[monetary_rew["Монетарный обзор по банкам"] != "Кредиты"]


In [7]:
def common_clean_data(df, delete_second_rows):
    df = df.copy()
    df.replace('-', np.nan, inplace=True)
    if delete_second_rows:
         df = pd.concat([df.iloc[:, 0], df.iloc[:, 1::2]], axis=1)
    df.columns = df.iloc[1]
    df.columns.values[0] = 'value'
    df.drop(index=1, inplace=True)
    df.dropna(how='any', inplace=True)
    df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda row: pd.to_numeric(row, errors='raise'))
    df.reset_index(drop=True, inplace=True)
    return df

In [8]:
dep_rates_cleaned = common_clean_data(dep_rates, True)
monetary_rew_cleaned = common_clean_data(monetary_rew, False)
money_mass_cleaned = common_clean_data(money_mass, False)

In [9]:
def to_long(df):
    return df.melt(id_vars="value",var_name="month",value_name="rate")

dep_rates_long = to_long(dep_rates_cleaned)
monetary_rew_long = to_long(monetary_rew_cleaned)  
money_mass_long = to_long(money_mass_cleaned)

In [10]:
context_df = pd.concat([dep_rates_long, monetary_rew_long, money_mass_long], ignore_index=True)

In [14]:
context_df['month'] = pd.to_datetime(
    context_df['month'].str.strip(),
    format='%m.%y'
)

context_df.head(50)

,value,month,rate
0,Депозиты небанковских юридических лиц,2022-01-01,7.9
1,до востребования,2022-01-01,0.1
2,условные,2022-01-01,4.3
3,"срочные, всего",2022-01-01,7.9
4,до 1 месяца,2022-01-01,7.9
5,от 1 до 3 месяцев,2022-01-01,8.2
6,от 3 месяцев до 1 года,2022-01-01,7.7
7,от 1 года до 5 лет,2022-01-01,7.9
8,свыше 5 лет,2022-01-01,9.7
9,Депозиты физических лиц,2022-01-01,8.4


In [12]:
context_df.to_csv('C:\\my\\prog\\pet banks\\etl\\context.csv', index=False)